In [31]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import PolynomialFeatures
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from joblib import dump, load
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import RidgeCV

In [32]:
df = pd.read_csv("insurance.csv")
df.head()

,age,sex,bmi,children,smoker,region,charges
0,19,female,27.900,0,yes,southwest,16884.92400
1,18,male,33.770,1,no,southeast,1725.55230
2,28,male,33.000,3,no,southeast,4449.46200
3,33,male,22.705,0,no,northwest,21984.47061
4,32,male,28.880,0,no,northwest,3866.85520


In [33]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1338 entries, 0 to 1337
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   age       1338 non-null   int64  
 1   sex       1338 non-null   object 
 2   bmi       1338 non-null   float64
 3   children  1338 non-null   int64  
 4   smoker    1338 non-null   object 
 5   region    1338 non-null   object 
 6   charges   1338 non-null   float64
dtypes: float64(2), int64(2), object(3)
memory usage: 73.3+ KB


In [34]:
df['sex'] = pd.get_dummies(df['sex'], dtype=int, drop_first=True)
df['smoker'] = pd.get_dummies(df['smoker'], dtype=int, drop_first=True)
df = pd.get_dummies(df, columns=['region'], prefix='region',dtype=int)

In [35]:
df.head()

,age,sex,bmi,children,smoker,charges,region_northeast,region_northwest,region_southeast,region_southwest
0,19,0,27.900,0,1,16884.92400,0,0,0,1
1,18,1,33.770,1,0,1725.55230,0,0,1,0
2,28,1,33.000,3,0,4449.46200,0,0,1,0
3,33,1,22.705,0,0,21984.47061,0,1,0,0
4,32,1,28.880,0,0,3866.85520,0,1,0,0


In [36]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1338 entries, 0 to 1337
Data columns (total 10 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   age               1338 non-null   int64  
 1   sex               1338 non-null   int32  
 2   bmi               1338 non-null   float64
 3   children          1338 non-null   int64  
 4   smoker            1338 non-null   int32  
 5   charges           1338 non-null   float64
 6   region_northeast  1338 non-null   int32  
 7   region_northwest  1338 non-null   int32  
 8   region_southeast  1338 non-null   int32  
 9   region_southwest  1338 non-null   int32  
dtypes: float64(2), int32(6), int64(2)
memory usage: 73.3 KB


In [37]:
#sns.heatmap(df.corr())

In [38]:
X = df.drop(['charges'], axis=1)
y = df['charges']

In [39]:
#sns.pairplot(data=df,diag_kind="kde")

In [40]:
polynomial_converter = PolynomialFeatures(degree=2, include_bias=False)
poly_X = polynomial_converter.fit_transform(X)

In [41]:
X_train, X_test, y_train, y_test = train_test_split(poly_X, y, test_size=0.3, random_state=101)

In [42]:
scaler = StandardScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

In [43]:
ridge_cv_model = RidgeCV(alphas=(0.1, 1.0, 1.05, 1.1, 1.15, 10.0))

In [44]:
ridge_cv_model.fit(X_train,y_train)

RidgeCV(alphas=(0.1, 1.0, 1.05, 1.1, 1.15, 10.0))

In [45]:
ridge_cv_model.alpha_

1.1

In [46]:
y_pred = ridge_cv_model.predict(X_test)

In [47]:
MAE = mean_absolute_error(y_true=y_test,y_pred=y_pred)
MAE

2892.735763099969

In [48]:
MSE = mean_squared_error(y_true=y_test,y_pred=y_pred)
MSE

22834882.87372522

In [49]:
RMSE = np.sqrt(MSE)
RMSE

4778.585865475812

In [50]:
tolerance = RMSE / y.mean()
tolerance

0.3600929774501751

In [51]:
r2_score = r2_score(y_true=y_test,y_pred=y_pred)
r2_score

0.837447734255824

In [52]:
ridge_cv_model.score(X_test,y_test)

0.837447734255824

In [53]:
# # Step 8/9 : choose right degree (3) Save model after we get OK for this result

final_poly_convertor = PolynomialFeatures(degree=2,include_bias=False)
final_model = RidgeCV(alphas=1.1)
poly_X = final_poly_convertor.fit_transform(X)
final_model.fit(poly_X,y)

RidgeCV(alphas=1.1)

In [54]:
dump(final_model,"insurance_ridge_cv_fin_2024_01_30.joblib")

['insurance_ridge_cv_fin_2024_01_30.joblib']

In [55]:
dump(final_poly_convertor,"insurance_converter_cv_fin_2024_01_30.joblib")

['insurance_converter_cv_fin_2024_01_30.joblib']

In [56]:
loaded_model = load("insurance_ridge_cv_fin_2024_01_30.joblib")
loaded_conv = load("insurance_converter_cv_fin_2024_01_30.joblib")

In [57]:
df.head(10)

,age,sex,bmi,children,smoker,charges,region_northeast,region_northwest,region_southeast,region_southwest
0,19,0,27.900,0,1,16884.92400,0,0,0,1
1,18,1,33.770,1,0,1725.55230,0,0,1,0
2,28,1,33.000,3,0,4449.46200,0,0,1,0
3,33,1,22.705,0,0,21984.47061,0,1,0,0
4,32,1,28.880,0,0,3866.85520,0,1,0,0
5,31,0,25.740,0,0,3756.62160,0,0,1,0
6,46,0,33.440,1,0,8240.58960,0,0,1,0
7,37,0,27.740,3,0,7281.50560,0,1,0,0
8,37,1,29.830,2,0,6406.41070,1,0,0,0
9,60,0,25.840,0,0,28923.13692,0,1,0,0


In [58]:
test_0 = [[19,0,27.9,0,1,0,0,0,1]]
test_2 = [[28,1,33,3,0,0,0,1,0]]
test_3 = [[33,1,22.705,0,0,0,1,0,0]]
test_4 = [[32,1,28.880,0,0,0,1,0,0]]
test_9 = [[60,0,25.840,0,0,1,0,0,0]]
test_poly = loaded_conv.transform(test_0)
test_poly
result = loaded_model.predict(test_poly)
print(result)
print(result/df['charges'].iloc[0])
print(result-df['charges'].iloc[0])

[24594.06987219]
[1.45656977]
[7709.14587219]


c:\Users\ericb\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but PolynomialFeatures was fitted with feature names
  warnings.warn(
